## Bonds and Interest rates
Ismael Oulkhir


Mail: oulkhir.ismael@gmail.com

LinkedIn: https://www.linkedin.com/in/ismail-oulkhir/


In [4]:
import numpy as np 
from numpy.random import normal
from scipy.stats import norm, gaussian_kde
import pandas as pd 
import plotly.graph_objects as go


## Varsicek Model
The Varsicek model is a model with : 
- Linear dynamics : drift is linear in r and that σ is constant as a function of r 
- Mean reversion capacity i.e the short rate has a tendency to revert to some long-term (possibly time-dependent) mean.


\begin{equation*}
dr_t = (b -ar_t )\, dt + \sigma dW_t , (a>0)
\end{equation*}



Where:
- $ r_t $ is the short rate at time $ t $,
- $ \sigma $ is the variance  at time $ t $,
- $ (b -ar_t ) $ is the drift rate of the short rate,

In [6]:
# Varsicek Model 
def varsicek(r0,a,b,T,sigma, paths):

    steps = round(252*T)
    dt = 1/steps
    r = np.zeros((steps+1, paths))
    r[0] = r0
    for t in range(1, 1+steps):
        dW_S = normal(0, np.sqrt(dt), paths)
        r[t] = r[t-1] + (b-a*r[t-1])*dt + sigma*dW_S

    return r

In [7]:
# Brownian Motion
def BM(r0,a,b,T,sigma, paths):
    steps = round(252*T)
    dt = 1/steps
    r = np.zeros((steps+1, paths))
    r[0] = r0

    for t in range(1,steps+1):
        dW = normal(0, np.sqrt(dt), paths)
        r[t] = r[t-1] + b/a* dt+ sigma * r[t-1] * dW
    return r

In [8]:
# -------SIMULATION OF BM AND Varsicek MODEL------- 

r0 = 4.30        # Initial stock price
T = 1           # Time to maturity (years)
a= 1.0 # speed of reversion parameter
b= 3.50  # Volatility of volatility
paths = 100000  # Number of simulations
sigma = 0.2     # Constant volatility for BM

# Simulation of varsicek Model and BM
r_varsicek = varsicek(r0,a,b,T,sigma, paths)
S_BM = BM(r0,a,b,T,sigma, paths)
# -------OPTION PRICING AND VOLATILITY SMILE-------




# Time axis
time = np.linspace(0, T, r_varsicek.shape[0])

# Plot Vasicek Model
fig_varsicek = go.Figure()
for i in range(10):  # Plot only the first 10 paths for clarity
    fig_varsicek.add_trace(go.Scatter(
        x=time,
        y=r_varsicek[:, i],
        mode='lines',
        name=f'Path {i+1}'
    ))
fig_varsicek.update_layout(
    title='Vasicek Model: Simulated Interest Rate Paths',
    xaxis_title='Time (Years)',
    yaxis_title='Interest Rate',
    showlegend=True
)
fig_varsicek.show()

# Plot GBM
fig_gbm = go.Figure()
for i in range(10):  # Plot only the first 10 paths for clarity
    fig_gbm.add_trace(go.Scatter(
        x=time,
        y=S_GBM[:, i],
        mode='lines',
        name=f'Path {i+1}'
    ))
fig_gbm.update_layout(
    title='Geometric Brownian Motion: Simulated Asset Price Paths',
    xaxis_title='Time (Years)',
    yaxis_title='Asset Price',
    showlegend=True
)
fig_gbm.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [5]:
# Brownian Motion PDF (log prices)
def GBMpdf(x, S0, r, sigma, T):

    loc = np.log(S0) + (r - 0.5 * sigma**2)*T
    scale = sigma * np.sqrt(T)
    pdf = norm.pdf(x, loc, scale)
    
    return pdf

# Heston PDF (log_prices)
def HestonPDF(x, S_T):
    kde = gaussian_kde(np.log(S_T))
    pdf = kde(x)
    return pdf

In [6]:
# Put option price (monte carlo approach)
def put_option_price(S_T, K, r, T):
    payoffs = np.maximum(K - S_T, 0)
    return np.mean(payoffs*np.exp(-r*T))

# Put option price (Black-Scholes approach)
def bs_put_price(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)


In [7]:
# Implied volatility calculation
def implied_vol(S0, K, T, r, option_price, tol=1e-5, max_iter=100):
    sigma = 0.2
    
    for _ in range(max_iter):
        price = bs_put_price(S0, K, T, r, sigma)
        vega = S0 * norm.pdf((np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))) * np.sqrt(T)
        diff = price - option_price

        if abs(diff) < tol:
            return sigma
        sigma -= diff / vega

    return sigma